In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sqlite3

In [3]:
root_dir = os.path.join("drive","MyDrive","courses","cs685","project","rat-sql")

In [4]:
!pip install asdl~=0.1.5 astor~=0.8.1 attrs~=18.2.0 babel~=2.7.0 bpemb~=0.2.11 cython~=0.29.1 jsonnet~=0.14.0 networkx~=2.2 nltk~=3.4 
!pip install numpy~=1.16 pyrsistent~=0.14.9 pytest~=5.3.2 records~=0.5.3 tabulate~=0.8.6 tqdm~=4.48.0
!pip install transformers -U 
!pip install sciket-learn
!pip install ent entmax stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement sciket-learn (from versions: none)
ERROR: No matching distribution found for sciket-learn
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import torch
print(torch.__version__)

2.0.0+cu118


In [6]:
import stanza

In [7]:
from transformers import RobertaModel,RobertaTokenizer 

import nltk

nltk.download('punkt')
nltk.download('stopwords')
model = RobertaModel.from_pretrained('roberta-large')
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
os.chdir(root_dir)

In [9]:
import json
import os
import _jsonnet
from ratsql.commands.infer import Inferer
from ratsql.datasets.spider import SpiderItem
from ratsql.utils import registry
import torch


In [10]:
import pandas as pd

In [11]:
exp_config_path = os.path.join(os.getcwd(),"experiments/spider-bert-run.jsonnet")
model_dir = os.path.join(os.getcwd(),"logdir","bert_run","bs=4,lr=1.0e-04,bert_lr=1.0e-05,end_lr=0e0,att=1")
print("model_dir={}".format(model_dir))
print("check if path exists = {}".format(os.path.exists(model_dir)))
checkpoint_step = 35100

model_dir=/content/drive/MyDrive/courses/cs685/project/rat-sql/logdir/bert_run/bs=4,lr=1.0e-04,bert_lr=1.0e-05,end_lr=0e0,att=1
check if path exists = True


In [12]:
root_dir = os.getcwd()
exp_config = json.loads(_jsonnet.evaluate_file(exp_config_path))
model_config_path = os.path.join(root_dir, exp_config["model_config"])
model_config_args = exp_config.get("model_config_args")
infer_config = json.loads(_jsonnet.evaluate_file(model_config_path, tla_codes={'args': json.dumps(model_config_args)}))
print(" ### INFER CONFIG ###")
print(infer_config)

inferer = Inferer(infer_config)
inferer.device = torch.device("cpu")
model = inferer.load_model(model_dir, checkpoint_step)
dataset = registry.construct('dataset', inferer.config['data']['val'])
print(" ######### DATASET #######")
print(dataset)

for id, schema in dataset.schemas.items():
    model.preproc.enc_preproc._preprocess_schema(schema)

 ### INFER CONFIG ###
{'data': {'test': {'db_path': 'data/spider/database', 'name': 'spider', 'paths': ['data/spider/test.json'], 'tables_paths': ['data/spider/tables.json']}, 'train': {'db_path': 'data/spider/database', 'name': 'spider', 'paths': ['data/spider/train_spider.json', 'data/spider/train_others.json'], 'tables_paths': ['data/spider/tables.json']}, 'val': {'db_path': 'data/spider/database', 'name': 'spider', 'paths': ['data/spider/dev.json'], 'tables_paths': ['data/spider/tables.json']}}, 'log': {'reopen_to_flush': True}, 'lr_scheduler': {'decay_steps': 35000, 'end_lr': 0, 'name': 'bert_warmup_polynomial_group', 'num_warmup_steps': 5000, 'power': 0.5, 'start_lr': 0.001, 'start_lrs': [0.0001, 1e-05]}, 'model': {'decoder': {'desc_attn': 'mha', 'dropout': 0.20687225956012834, 'enc_recurrent_size': 1024, 'loss_type': 'softmax', 'name': 'NL2Code', 'recurrent_size': 512, 'use_align_loss': True, 'use_align_mat': True}, 'decoder_preproc': {'grammar': {'clause_order': None, 'end_with

WARNING <class 'ratsql.models.enc_dec.EncDecModel.Preproc'>: superfluous {'name': 'EncDec'}
WARNING <class 'ratsql.models.enc_dec.EncDecModel'>: superfluous {'decoder_preproc': {'grammar': {'clause_order': None, 'end_with_from': True, 'factorize_sketch': 2, 'include_literals': True, 'infer_from_conditions': True, 'name': 'spider', 'output_from': True, 'use_table_pointer': True}, 'save_path': 'data/spider/nl2code,output_from=true,fs=2,emb=bert,cvlink', 'use_seq_elem_rules': True}, 'encoder_preproc': {'bert_version': 'roberta-large', 'compute_cv_link': True, 'compute_sc_link': True, 'db_path': 'data/spider/database', 'fix_issue_16_primary_keys': True, 'include_table_name_in_column': False, 'save_path': 'data/spider/nl2code,output_from=true,fs=2,emb=bert,cvlink'}}
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_h

Loading model from /content/drive/MyDrive/courses/cs685/project/rat-sql/logdir/bert_run/bs=4,lr=1.0e-04,bert_lr=1.0e-05,end_lr=0e0,att=1/model_checkpoint-00035100
{'model': EncDecModel(
  (encoder): SpiderEncoderBert(
    (encs_update): RelationalTransformerUpdate(
      (encoder): Encoder(
        (layers): ModuleList(
          (0-7): 8 x EncoderLayer(
            (self_attn): MultiHeadedAttentionWithRelations(
              (linears): ModuleList(
                (0-3): 4 x Linear(in_features=1024, out_features=1024, bias=True)
              )
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (feed_forward): PositionwiseFeedForward(
              (w_1): Linear(in_features=1024, out_features=4096, bias=True)
              (w_2): Linear(in_features=4096, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (sublayer): ModuleList(
              (0-1): 2 x SublayerConnection(
                (norm): 

DB connections: 100%|██████████| 166/166 [00:04<00:00, 36.54it/s]
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


 ######### DATASET #######


INFO:stanza:Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| lemma     | combined |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: lemma


INFO:stanza:Done loading processors!


In [13]:
def question(q, db_id):
    spider_schema = dataset.schemas[db_id]
    colMap = {}
    data_item = SpiderItem(
        text=None,  # intentionally None -- should be ignored when the tokenizer is set correctly
        code=None,
        schema=spider_schema,
        orig_schema=spider_schema.orig,
        orig={"question": q}
    )
    model.preproc.clear_items()
    enc_input = model.preproc.enc_preproc.preprocess_item(data_item, None)
    preproc_data = enc_input, None
    with torch.no_grad():
        res, _, _  = inferer._infer_one(model, data_item, preproc_data, beam_size=1, use_heuristic=True)
        return res

In [14]:
def check_datatype(val):
    # if isinstance(val,tuple) or isinstance(val,list):
    #     if len(val)>1:
    #         return val
    #     else:
    #         return val[0]    
    # else:
    #     return val
    return val

def run_query(db_id, query):
    root_path = os.getcwd()
    data_dir = os.path.join(root_path, "data","spider", "database",f"{db_id}")
    db_path = os.path.join(data_dir,f'{db_id}.sqlite')
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Execute an SQL query
    cursor.execute(query)

    # Fetch the results
    results = cursor.fetchall()
    res = []
    # Process the results
    for row in results:
        res.append(check_datatype(row))
    # Close the database connection
    conn.close()
    return res

In [15]:
df_path = os.path.join(os.getcwd(),"..","final_set_questions_with_results.csv")
final_df = pd.read_csv(df_path)

In [16]:
db_list = list(final_df['database_id'])
utterance_list = list(final_df['utterance'])
query_list = list(final_df['query'])
utterance_idx = list(final_df['utterance_idx'])
interaction_idx = list(final_df['interaction_idx'])
finalqn_results = list(final_df['utterance_result'])

In [17]:
predicted_query = []
predicted_query_res = []
for utterance,query,db_id in zip(utterance_list, query_list, db_list):
  res = question(utterance, db_id)
  pred_query = res[0]['inferred_code']
  pred_query_res = run_query(db_id,pred_query)
  print(pred_query, pred_query_res)
  predicted_query.append(pred_query)
  predicted_query_res.append(pred_query_res)

SELECT airlines.Country FROM airlines WHERE airlines.Airline = 'terminal' []
SELECT airlines.Abbreviation FROM airlines WHERE airlines.Airline = 'terminal' []
SELECT airlines.Airline, airlines.Abbreviation FROM airlines WHERE airlines.Country = 'terminal' []
SELECT airports.AirportCode, airports.AirportName FROM airports WHERE airports.City = 'terminal' []
SELECT Count(*) FROM airlines [(12,)]
SELECT Count(*) FROM airports WHERE airports.AirportName = 'terminal' [(0,)]
SELECT Count(*) FROM flights [(1200,)]
SELECT * FROM airlines WHERE airlines.Abbreviation = 'terminal' []
SELECT Count(*) FROM airlines WHERE airlines.Country = 'terminal' [(0,)]
SELECT airports.City, airports.Country FROM airports WHERE airports.AirportName = 'terminal' []
SELECT airports.AirportName FROM airports WHERE airports.AirportCode = 'terminal' []
SELECT airports.AirportName FROM airports WHERE airports.City = 'terminal' []
SELECT Count(*) FROM airports JOIN flights ON airports.AirportCode = flights.SourceAirpo

In [20]:
final_ques_df_dict = {'database_id':db_list,
                 'utterance': utterance_list,
                 'query': query_list,
                 'utterance_idx': utterance_idx,
                 'interaction_idx': interaction_idx,
                 'finalqn_results': finalqn_results,
                 'predicted_query': predicted_query,
                 'predicted_query_res': predicted_query_res
                 }

In [21]:
final_ques_df = pd.DataFrame.from_dict(final_ques_df_dict)
op_path = os.path.join(os.getcwd(),"..","final_set_questions_with_results_and_predictions.csv")
final_ques_df.to_csv(op_path)